In [11]:
#/usr/bin/env python

# Libraries to query D2P2 API and process the results
import json
import urllib
import requests

# general utility libraries
import re
import os
import numpy
import pandas as pd

# Biopython related libraries for sequences analysis
import Bio
from Bio import Entrez
from Bio import AlignIO
from Bio.Align.Applications import ClustalwCommandline
from Bio.pairwise2 import format_alignment
from Bio import SeqIO
from Bio import pairwise2

# Query the human proteome against the D2P2 database and store disordered sequences, PFAM domains, and list of PFAM information

## Use only if you want to generate all the raw data from source. This takes ~ 4.5 hours to run ~20,000 API requests at roughly 1 request/second

## The input_filename should be input fasta file for which you want to query disorder+pfams

### Currently, the data that is stored is the consensus disorder, the PFAM (excluding PFAM-b) domains and their positions, 

In [3]:
filename = "../data/raw_sequence_data/nuclear_proteins.fasta"

filename_write_data = "../data/raw_sequence_data/nuclear_proteins_pfam_domain_and_disorder.tsv"

if not os.path.exists(os.path.dirname(filename_write_data)):
    try:
        os.makedirs(os.path.dirname(filename_write_data))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise


records = list(SeqIO.parse(filename,"fasta"))

output_data = {}

with open(filename_write_data,'a+') as file_write:

    for item in records:

        Id_current = str(item.id.split('|')[1])
        gene_name = str(item.id.split('|')[2])
        description = str(item.description.split('|')[-1])
        seq = str(item.seq);
        data = 'seqids=["{}"]'.format(Id_current)
        request = requests.get('http://d2p2.pro/api/seqid', data)
        response = request.json()
        ID_list = [];

        if (response[Id_current]):

            count = 1;
            for pos in response[Id_current][0][2]['disorder']['consranges']:
                Id = Id_current + '_' + str(count);
                output_data[Id] = {};
                output_data[Id]['id'] = Id_current;
                output_data[Id]['seq'] = seq;
                output_data[Id]['description'] = description;
                output_data[Id]['idr_start'] = pos[0];
                output_data[Id]['idr_end'] = pos[1]; 
                output_data[Id]['gene_name'] = gene_name; 
            
                ID_list.append(Id)
                count = count+1;

            count_dom = 0;

            if count==1:
                output_data[Id_current] = {};
                output_data[Id_current]['id'] = Id_current;
                output_data[Id_current]['seq'] = seq;
                output_data[Id_current]['description'] = description;
                output_data[Id_current]['idr_start'] = -1;
                output_data[Id_current]['idr_end'] = -1;
                output_data[Id_current]['gene_name'] = gene_name; 

                ID_list.append(Id_current)

            count_dom = 1;

            id_domains = {};
            for pfams in (response[Id_current][0][2]['structure']['pfam']):
                pfam_name = pfams[2][0:7];

                if (pfam_name.find('PF') > -1) :

                    domain_id = Id_current + '_pfam_' + str(count_dom);
                    id_domains[domain_id] = {};
                    id_domains[domain_id]['id'] = pfam_name;
                    id_domains[domain_id]['start'] = int(pfams[7])
                    id_domains[domain_id]['end'] = int(pfams[8])
                    id_domains[domain_id]['escore'] = float(pfams[5])
                    id_domains[domain_id]['pfam_name'] = str(pfams[3])
                    id_domains[domain_id]['pfam_desc'] = str(pfams[4])
                    count_dom = count_dom + 1;

            if count_dom ==1:
                domain_id = Id_current ;
                id_domains[domain_id] = {};
                id_domains[domain_id]['id'] = 'None';

            for ID in ID_list:
                output_data[ID]['pfam_list'] = [];
                for dom_id in list(id_domains.keys()):
                    str_to_write = '';
                    for key in id_domains[dom_id].keys():
                        str_to_write = str_to_write + str(id_domains[dom_id][key]) + '_';
                    output_data[ID]['pfam_list'].append(str_to_write)


        else:
            output_data[Id_current] = {};
            output_data[Id_current]['id'] = Id_current;
            output_data[Id_current]['seq'] = seq;
            output_data[Id_current]['description'] = description;
            output_data[Id_current]['idr_start'] = -1;
            output_data[Id_current]['idr_end'] = -1;
            output_data[Id_current]['gene_name'] = gene_name; 
            output_data[Id_current]['pfam_list'] = ['None'];
            
            ID_list.append(Id_current);
            
        for ID in ID_list:
            data_to_write = list(output_data[ID].values());
            file_write.write(str(ID)+'\t');
            for data in data_to_write:
                if not isinstance(data, list):
                    file_write.write(str(data) + '\t');
                else:
                    for pfam in data:
                        file_write.write(str(pfam) + '\t');
                        
            file_write.write('\n')


## Save file containing PFAM domains and IDRs into a sorted file

In [13]:
# Input
data_file = "../data/raw_sequence_data/nuclear_proteins_pfam_domain_and_disorder.tsv"

# Delimiter
data_file_delimiter = '\t'

# The max column count a line in the file could have
largest_column_count = 0

# Loop the data lines
with open(data_file, 'r') as temp_f:
    # Read the lines
    lines = temp_f.readlines()

    for l in lines:
        # Count the column count for the current line
        column_count = len(l.split(data_file_delimiter)) + 1
        
        # Set the new most column count
        largest_column_count = column_count if largest_column_count < column_count else largest_column_count

# Generate column names (will be 0, 1, 2, ..., largest_column_count - 1)
column_names = [i for i in range(0, largest_column_count)]

# Read csv
df = pd.read_csv(data_file, header=None, delimiter=data_file_delimiter, names=column_names)
# print(df)

/home/npradeep96/miniconda3/envs/IDRBlocks/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
p = df.columns[7:]
df['pfam_all'] = df[p].astype(str).apply(lambda x: ','.join(x), axis=1)
for count in range(len(df['pfam_all'].values)):
    df['pfam_all'].values[count] = df['pfam_all'].values[count].strip('nan,')
less_data = df[list(df.columns[0:7])+ ['pfam_all']]
less_data.columns = ['IDR_id','ID','sequence','description','idr_start','idr_end','gene','pfam_all']
less_data

,IDR_id,ID,sequence,description,idr_start,idr_end,gene,pfam_all
0,Q5JTC6_1,Q5JTC6,METQKDEAAQAKGAAASGSTREQTAEKGAKNKAAEATEGPTSEPSS...,AMER1_HUMAN APC membrane recruitment protein 1...,1,57,AMER1_HUMAN,PF09422_88_539_3.1000000000000004e-167_None_None_
1,Q5JTC6_2,Q5JTC6,METQKDEAAQAKGAAASGSTREQTAEKGAKNKAAEATEGPTSEPSS...,AMER1_HUMAN APC membrane recruitment protein 1...,71,309,AMER1_HUMAN,PF09422_88_539_3.1000000000000004e-167_None_None_
2,Q5JTC6_3,Q5JTC6,METQKDEAAQAKGAAASGSTREQTAEKGAKNKAAEATEGPTSEPSS...,AMER1_HUMAN APC membrane recruitment protein 1...,330,412,AMER1_HUMAN,PF09422_88_539_3.1000000000000004e-167_None_None_
3,Q5JTC6_4,Q5JTC6,METQKDEAAQAKGAAASGSTREQTAEKGAKNKAAEATEGPTSEPSS...,AMER1_HUMAN APC membrane recruitment protein 1...,420,486,AMER1_HUMAN,PF09422_88_539_3.1000000000000004e-167_None_None_
4,Q5JTC6_5,Q5JTC6,METQKDEAAQAKGAAASGSTREQTAEKGAKNKAAEATEGPTSEPSS...,AMER1_HUMAN APC membrane recruitment protein 1...,516,517,AMER1_HUMAN,PF09422_88_539_3.1000000000000004e-167_None_None_
...,...,...,...,...,...,...,...,...
42762,F8W7U8_1,F8W7U8,MSTADALDDENTFKILVATDIHLGFMEKDAVRGNDTFVTLDEILRL...,F8W7U8_HUMAN Double-strand break repair protei...,502,532,F8W7U8_HUMAN,PF00149_13_249_1.4000000000000001e-30_Metallop...
42763,F8W7U8_2,F8W7U8,MSTADALDDENTFKILVATDIHLGFMEKDAVRGNDTFVTLDEILRL...,F8W7U8_HUMAN Double-strand break repair protei...,535,612,F8W7U8_HUMAN,PF00149_13_249_1.4000000000000001e-30_Metallop...
42764,F8W7U8_3,F8W7U8,MSTADALDDENTFKILVATDIHLGFMEKDAVRGNDTFVTLDEILRL...,F8W7U8_HUMAN Double-strand break repair protei...,633,707,F8W7U8_HUMAN,PF00149_13_249_1.4000000000000001e-30_Metallop...
42765,H3BTM3,H3BTM3,MAEAGPQAPPPPGTPSRHEKSLGLLTTKFVSLLQEAKDGVLDLKLA...,H3BTM3_HUMAN Transcription factor E2F4 OS=Homo...,-1,-1,H3BTM3_HUMAN,None


In [18]:
print(p)

Int64Index([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
            24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
            41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57,
            58, 59, 60, 61, 62, 63, 64, 65],
           dtype='int64')


# collapse all pfam files into one column, all idrs of a given protein into one column, and output sorted data-frame (by ID) to write_file

In [ ]:
p = df.columns[7:121]
df['pfam_all'] = df[p].astype(str).apply(lambda x: ','.join(x), axis=1)
for count in range(len(df['pfam_all'].values)):
    df['pfam_all'].values[count] = df['pfam_all'].values[count].strip('nan,')
less_data = df[list(df.columns[0:7])+ ['pfam_all']]
less_data.columns = ['IDR_id','ID','sequence','description','idr_start','idr_end','gene','pfam_all']
less_data

In [22]:
sorted_data = less_data.astype(str).groupby('ID').agg(lambda x: ','.join(x.unique()));
length_proteins = numpy.array( [float(len(x)) for x in sorted_data.iloc[:,1]])
sorted_data['lengths'] = length_proteins;
sorted_data

,IDR_id,sequence,description,idr_start,idr_end,gene,pfam_all,lengths
ID,,,,,,,,
A0A024R0Y4,"A0A024R0Y4_1,A0A024R0Y4_2,A0A024R0Y4_3",MDRLGSFSNDPSDKPPCRGCSSYLMEPYIKCAECGPPPFFLCLQCF...,A0A024R0Y4_HUMAN Transcriptional adapter OS=Ho...,"4,142,348","6,149,371",A0A024R0Y4_HUMAN,PF00249_72_118_1.1e-09_Myb_DNA-binding_Myb-lik...,443.0
A0A024R7E8,A0A024R7E8,MVRSRLTAVSASWVQAHPPADMGRRKSKRKPPPKKKMTGTLETQFT...,A0A024R7E8_HUMAN Transcription elongation fact...,-1,-1,A0A024R7E8_HUMAN,None,104.0
A0A024RA52,A0A024RA52,MAERGYSFSLTTFSPSGKLVQIEYALAAVAGGAPSVGIKAANGVVL...,A0A024RA52_HUMAN Proteasome subunit alpha type...,-1,-1,A0A024RA52_HUMAN,PF10584_6_28_5.4e-11_Proteasome_A_N_Proteasome...,234.0
A0A024RAC6,A0A024RAC6,MAAESALQVVEKLQARLAANPDPKKLLKYLKKLSTLPITVDILAET...,A0A024RAC6_HUMAN Elongin-A OS=Homo sapiens OX=...,-1,-1,A0A024RAC6_HUMAN,None,772.0
A0A024RCN4,A0A024RCN4,MATALVSAHSLAPLNLKKEGLRVVREDHYSTWEQGFKLQGNSKGLG...,A0A024RCN4_HUMAN Zinc finger and SCAN domain-c...,-1,-1,A0A024RCN4_HUMAN,None,479.0
...,...,...,...,...,...,...,...,...
X6R2L4,X6R2L4,MATVTATTKVPEIRDVTRIERIGAHSHIRGLGLDDALEPRQASQGM...,X6R2L4_HUMAN RuvB-like helicase OS=Homo sapien...,-1,-1,X6R2L4_HUMAN,None,259.0
X6R549,X6R549,RPRRGGAGTGPRGESAGLLLLLLQDALPRSGLNLKEEPLLPAGLGS...,X6R549_HUMAN Transcription factor COE4 (Fragme...,-1,-1,X6R549_HUMAN,None,621.0
X6R7X0,"X6R7X0_1,X6R7X0_2,X6R7X0_3,X6R7X0_4,X6R7X0_5",MAVFSGVLESALFLDQNTLQARQGSRSPLTMDKFVIRTPRIQNSPQ...,X6R7X0_HUMAN Transcription elongation factor A...,"38,40,145,155,163","38,53,148,158,163",X6R7X0_HUMAN,PF08711_91_142_5e-17_Med26_TFIIS helical bundl...,238.0


In [23]:
write_sorted_data ='raw_data/20190709_clean_output_test_repeat.csv'
sorted_data.to_excel(write_sorted_data)

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/20190709_clean_output_test_repeat.xlsx'